 #  LangChain Hands-on

### Content
       - LLM Environment
       - Chat Models: GPT-3.5 Turbo and GPT-4
       - Caching : In-Memory and SQL lite 
       - Streaming
       - Prompt Templates and Chat Prompt Templates
       - Chains : Simple and Sequential
       - Langchain Agents in Action : Pyhton REPL
       - LangChain Search Tools: DuckDuckGo and Wikipedia

In [4]:
# pip install -r ./requirements.txt -q

Download [requirements.txt](https://drive.google.com/file/d/1UpURYL9kqjXfe9J8o-_Dq5KJTbQpzMef/view?usp=sharing)

In [5]:
 # pip install --upgrade -q langchain

In [6]:
 # pip install --upgrade -q openai

In [7]:
# pip show openai

In [8]:
# pip show langchain

### Python-dotenv

In [9]:
# pip install python-dotenv

In [10]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

#os.environ.get('OPENAI_API_KEY')

True

## Chat Models: GPT-3.5 Turbo and GPT-4

In [32]:
#pip install pydantic==1.10.8

In [26]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI()  

# invoking the llm (running the prompt) 
# Temperature is a parameter that controls the “creativity” or randomness of the text generated by GPT-3
output = llm.invoke('Explain NEET Controversy in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

The NEET controversy revolves around the fairness and accessibility of the National Eligibility cum Entrance Test for medical admissions in India.


In [117]:
help(ChatOpenAI)  # see the llm constructor arguments with its defaults

Help on class ChatOpenAI in module langchain_openai.chat_models.base:

class ChatOpenAI(langchain_core.language_models.chat_models.BaseChatModel)
 |  ChatOpenAI(*, name: Union[str, NoneType] = None, cache: Union[bool, NoneType] = None, verbose: bool = None, callbacks: ForwardRef('Callbacks') = None, tags: Union[List[str], NoneType] = None, metadata: Union[Dict[str, Any], NoneType] = None, callback_manager: Union[langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, client: Any = None, async_client: Any = None, model: str = 'gpt-3.5-turbo', temperature: float = 0.7, model_kwargs: Dict[str, Any] = None, api_key: Union[pydantic.types.SecretStr, NoneType] = None, base_url: Union[str, NoneType] = None, organization: Union[str, NoneType] = None, openai_proxy: Union[str, NoneType] = None, timeout: Union[float, Tuple[float, float], Any, NoneType] = None, max_retries: int = 2, streaming: bool = False, n: int = 1, max_tokens: Union[int, NoneType] = None, tiktoken_model_name: Union

In [70]:
# using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(
    SystemMessage, 
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in Hindi.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

क्वांटम मैकेनिक्स एक भौतिकी क्षेत्र है जो अत्यंत छोटे रसायनिक क्रियाओं और ऊर्जाओं का अध्ययन करता है।


## Caching 

### 1. In-Memory Cache

In [71]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [72]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [73]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 2.58 s


'\n\nWhy did the cookie go to the doctor?\n\nBecause it was feeling crumbly!'

In [74]:
%%time
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 1.03 ms


'\n\nWhy did the cookie go to the doctor?\n\nBecause it was feeling crumbly!'

### 2. SQLite Caching

In [75]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [76]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: total: 0 ns
Wall time: 10.1 ms


"\n\nWhy don't scientists trust atoms? Because they make up everything."

In [77]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: total: 0 ns
Wall time: 3.36 ms


"\n\nWhy don't scientists trust atoms? Because they make up everything."

## LLM Streaming

In [79]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
Underneath the pale moonlight
A raven takes flight
Its black wings against the sky
As it soars up high

Chorus:
Fly, fly, raven in the night
Under the moon's soft light
A mysterious dance between the two
The moon and the raven, forever true

Verse 2:
The moon whispers secrets to the bird
As they circle above
A bond that can never be broken
A connection born of love

Chorus:
Fly, fly, raven in the night
Under the moon's soft light
A mysterious dance between the two
The moon and the raven, forever true

Bridge:
In the darkness they find solace
In each other's company
The moon and the raven, a timeless pair
Bound by destiny

Chorus:
Fly, fly, raven in the night
Under the moon's soft light
A mysterious dance between the two
The moon and the raven, forever true

Outro:
So when you see a raven in the sky
And the moon shining bright
Remember the bond they share
A love that lasts through the night.


In [80]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)# flush=True ensures that the output is immediately flushed and displayed on the screen

(Verse 1)
Underneath the silver light
The Raven takes to flight
Through the darkness of the night
He cries out with all his might

(Chorus)
Moon and Raven, dancing in the sky
Their spirits soaring high
A bond that cannot be denied
In the stillness of the night

(Verse 2)
The Moon, so bright and bold
Guiding the Raven as he unfolds
His wings spread wide, so free
Together they conquer the sea

(Chorus)
Moon and Raven, dancing in the sky
Their spirits soaring high
A bond that cannot be denied
In the stillness of the night

(Bridge)
The Raven calls out to the Moon
A haunting melody, a mournful tune
Their connection strong and true
In the darkness, they both renew

(Chorus)
Moon and Raven, dancing in the sky
Their spirits soaring high
A bond that cannot be denied
In the stillness of the night

(Outro)
In the midnight hour, they reign
A cosmic duo, never in vain
Moon and Raven, forever intertwined
In the vast expanse of time

## Prompt Templates

In [87]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced stock market expert.
Write a few sentences about the following stock "{stock}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(stock='Infosys', language='English')
prompt  # Returns the generated prompt


'You are an experienced stock market expert.\nWrite a few sentences about the following stock "Infosys" in English.'

In [88]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

Infosys is a multinational corporation that provides business consulting, information technology, and outsourcing services. It is one of the largest IT companies in India and has a strong presence in the global market. Infosys stock has shown consistent growth over the years, making it a popular choice among investors looking for stable returns in the technology sector. With a strong track record of innovation and customer satisfaction, Infosys is considered a reliable investment option for those looking to diversify their portfolio in the tech industry.


## Chat Prompt Templates

- The roles in this class are:

    - `System` for a system chat message setting the stage (e.g., “You are a knowledgeable historian”).
    - `User`, which contains the user’s specific historical question.
    
    - `AI`, which contains the LLM’s preliminary response or follow-up question.

In [96]:
from langchain_core.prompts import ChatPromptTemplate

# Define roles and placeholders
chat_template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a expert in HR department in data science company. You are called {name}."),
    ("user", "Hi, what's the mostly lieky {n} question HR asked during interview?"),
  ]
)

messages = chat_template.format_messages(name="Mike", n="5")
print(messages)

[SystemMessage(content='You are a expert in HR department in data science company. You are called Mike.'), HumanMessage(content="Hi, what's the mostly lieky 5 question HR asked during interview?")]


In [97]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

Hello! In data science company, HR typically asks the following questions during interviews:

1. Can you walk me through your experience with data science projects?
2. How do you stay updated with the latest trends and technologies in the field of data science?
3. Can you provide an example of a challenging problem you solved using data analysis?
4. How do you approach communication and collaboration with non-technical stakeholders?
5. What do you consider to be the most important skills for a successful data scientist and how do you continue to develop those skills?

These questions help HR assess the candidate's technical skills, problem-solving abilities, communication skills, and overall fit for the role in the data science company.


## Simple Chains

In [100]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'English'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in English.

> Finished chain.


In [101]:
print(output)

{'virus': 'HSV', 'language': 'English', 'text': "HSV, or herpes simplex virus, is a common virus that causes cold sores and genital herpes. There are two types of HSV: HSV-1, which typically causes cold sores on the mouth, and HSV-2, which typically causes genital herpes. HSV is highly contagious and can be spread through direct contact with an infected person's skin or bodily fluids. While there is no cure for HSV, antiviral medications can help manage symptoms and reduce the frequency of outbreaks."}


In [104]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific places
output = chain.invoke(country)
print(output['text'])

Enter Country: USA


> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of USA?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
The capital of the USA is Washington, D.C.

- The White House
- The National Mall
- The Smithsonian Institution


## Sequential Chains

In [108]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here is an example of a Python function that implements linear regression:

```python
import numpy as np

def linear_regression(x, y):
    n = len(x)
    
    # Calculate the mean of x and y
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    
    # Calculate the terms needed for the slope and intercept
    numer = 0
    denom = 0
    for i in range(n):
        numer += (x[i] - mean_x) * (y[i] - mean_y)
        denom += (x[i] - mean_x) ** 2
    
    # Calculate the slope and intercept
    slope = numer / denom
    intercept = mean_y - slope * mean_x
    
    return slope, intercept

# Example usage
x = np.array([1, 2, 3, 4, 5])
y = np.array([2, 3, 4, 5, 6])

slope, intercept = linear_regression(x, y)
print(f"Slope: {slope}, Intercept: {intercept}")
```

This function takes two arrays `x` and `y` as input, which represent the independent and dependent variables, respectively. It calculates the slope and intercept of the linear regres

In [109]:
print(output['output'])

The provided Python function, `linear_regression(x, y)`, is designed to calculate the parameters of a linear regression model (specifically, the slope and intercept) that best fits a set of data points. The code follows a straightforward and traditional approach to linear regression, using only numpy for basic mathematical operations and assuming a model of the form `y = slope * x + intercept`. Here's a step-by-step breakdown of how the function works:

1. **Input arrays**: The function accepts two numpy arrays as input. The array `x` represents the predictor variable(s), and `y` contains the dependent variable corresponding to each `x` value. These arrays should be of the same length since each `x[i]` corresponds to a `y[i]`.

2. **Initialize variables**: The function starts by calculating the number of data points (`n`) using the `len(x)` function. This `n` is used in loops and in calculating mean values.

3. **Calculate the means**: It computes the mean of the `x` values (`mean_x`) 

## Langchain Agents in Action : Pyhton REPL

In [57]:
# pip install -q langchain_experimental

In [111]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To solve this, I will first calculate the factorial of 12 using the `math` module's `factorial` function. Then, I will calculate the square root of that result using the `sqrt` function from the same module. Finally, I will format the result to display it with 4 decimal points using the `format` function.

Action: Python_REPL
Action Input: 
import math
result = math.sqrt(math.factorial(12))
formatted_result = format(result, '.4f')
print(formatted_result)
Observation: 21886.1052

Thought:I now know the final answer
Final Answer: 21886.1052

> Finished chain.


{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [112]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [113]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [114]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [115]:
print(response['output'])


146306.05007233328


## LangChain Tools: DuckDuckGo and Wikipedia

In [11]:
#pip install -q duckduckgo-search

In [12]:
#pip install --upgrade --quiet duckduckgo-search

In [13]:
# optional, filter workings (recommended in production)
import warnings
warnings.filterwarnings('ignore', module='langchain')

In [14]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was Mahatma Gandhi born?')
print(output)

Mohandas Karamchand Gandhi (ISO: Mōhanadāsa Karamacaṁda Gāṁdhī; [c] 2 October 1869 - 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule.He inspired movements for civil rights and freedom across the world. Mahatma Gandhi (born October 2, 1869, Porbandar, India—died January 30, 1948, Delhi) was an Indian lawyer, politician, social activist, and writer who became the leader of the Indian Independence Movement against British rule. As such, he came to be considered the father of his country. Gandhi is internationally esteemed for his doctrine of ... Mahatma Gandhi - Indian Leader, Nonviolence, Activist: The British attitude toward Gandhi was one of mingled admiration, amusement, bewilderment, suspicion, and resentment. Except for a tiny minority of Christian missionaries and radical socialists, the British tended to see him at best as a utopian

In [15]:
search.name

'duckduckgo_search'

In [16]:
search.description

'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.'

In [22]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.tools import DuckDuckGoSearchResults

wrapper = DuckDuckGoSearchAPIWrapper(region='in-en', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Indian Election 2024 summary')

In [23]:
print(output)

[snippet: 2024 Indian general election. General elections were held in India from 19 April to 1 June 2024 in seven phases, to elect all 543 members of the Lok Sabha. [ a] Votes were counted and the result was declared on 4 June to form the 18th Lok Sabha. [ 2][ 3] On 7 June 2024, Prime Minister Narendra Modi confirmed the support of 293 MPs to Droupadi ..., title: 2024 Indian general election - Wikipedia, link: https://en.wikipedia.org/wiki/2024_Indian_general_election], [snippet: The 2024 Lok Sabha elections were a significant event in India's political landscape, marked by intense competition and unexpected outcomes. In this visual guide, we break down the key results, major upsets, and notable events that shaped the election. From vote counts to party performances, heres a graphic explanation of what happened during the 2024 Lok Sabha elections., title: Lok Sabha Elections 2024: Results and highlights explained in graphics, link: https://economictimes.indiatimes.com/news/elections/l

In [24]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

Snippet: 2024 Indian general election. General elections were held in India from 19 April to 1 June 2024 in seven phases, to elect all 543 members of the Lok Sabha. [ a] Votes were counted and the result was declared on 4 June to form the 18th Lok Sabha. [ 2][ 3] On 7 June 2024, Prime Minister Narendra Modi confirmed the support of 293 MPs to Droupadi ...
Title: 2024 Indian general election - Wikipedia
Link: https://en.wikipedia.org/wiki/2024_Indian_general_election

--------------------------------------------------
Snippet: The 2024 Lok Sabha elections were a significant event in India's political landscape, marked by intense competition and unexpected outcomes. In this visual guide, we break down the key results, major upsets, and notable events that shaped the election. From vote counts to party performances, heres a graphic explanation of what happened during the 2024 Lok Sabha elections.
Title: Lok Sabha Elections 2024: Results and highlights explained in graphics
Link: https://e

### Wikipedia

In [ ]:
#pip install -q wikipedia

In [18]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [19]:
wiki.invoke({'query': 'Matplotlib'})

'Page: Matplotlib\nSummary: Matplotlib is a plotting library for the Python programming language and its numerical mathematics extension NumPy. It provides an object-oriented API for embedding plots into applications using general-purpose GUI toolkits like Tkinter, wxPython, Qt, or GTK. There is also a procedural "pylab" interface based on a state machine (like OpenGL), designed to closely resemble that of MATLAB, though its use is discouraged. SciPy makes use of Matplotlib.\nMatplotlib was originally written by John D. Hunter. Since then it has had an active development community and is distributed under a BSD-style license. Michael Droettboom was nominated as matplotlib\'s lead developer shortly before John Hunter\'s death in August 2012 and was further joined by Thomas Caswell. Matplotlib is a NumFOCUS fiscally sponsored project.'

In [20]:
output = wiki.invoke('Google Gemini')

In [21]:
print(output)

Page: Gemini (chatbot)
Summary: Gemini, formerly known as Bard, is a generative artificial intelligence chatbot developed by Google. Based on the large language model (LLM) of the same name and developed as a direct response to the meteoric rise of OpenAI's ChatGPT, it was launched in a limited capacity in March 2023 before expanding to other countries in May. It was previously based on PaLM, and initially the LaMDA family of large language models.
LaMDA had been developed and announced in 2021, but it was not released to the public out of an abundance of caution. OpenAI's launch of ChatGPT in November 2022 and its subsequent popularity caught Google executives off-guard and sent them into a panic, prompting a sweeping response in the ensuing months. After mobilizing its workforce, the company launched Bard in February 2023, which took center stage during the 2023 Google I/O keynote in May and was upgraded to the Gemini LLM in December. Bard and Duet AI were unified under the Gemini br